m9-2 . 加dropout.

两部分的输出用 fc+bn 层拼接。

- doc_len: 每个文档的句子数；
- sent_len: 每个句子的词数量；
- embedding_size: 词向量长度。

X_inputs.shape=[batch_size, doc_len, sent_len] ->(0.embedding)-> sent_inputs.shape=[batch_size, doc_len, sent_len, embedding_size] 

sentence_encoder:每个句子得到一个向量:<br/>
sent_inputs ->(reshape)-> [batch_size×doc_len, sent_len, embedding_size] ->(bigru+attention)-> doc_inputs:[batch_size×doc_len, hidden_size×2]

doc_encoder: 每个文档得到一个向量：<br/>
doc_inputs:[batch_size×doc_len, hidden_size×2] ->(reshpe)-> [batch_size, doc_len, hidden_size×2] ->(bigru+attention)-> doc_outputs:[batch_size, hidden_size*2]

sigmoig_output_layer.

参考代码： 

https://github.com/yongyehuang/deep-text-classifier/blob/master/HAN_model.py 其中attention最后加权的地方应该写错了

https://github.com/indiejoseph/doc-han-att/blob/master/model.py

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import time

time0 = time.time()
print('Starting ...')
model_name = 'm9-2-han-bigru-title-content-512-30'                    # 模型名称
W_embedding = np.load('../data/W_embedding.npy')            # 导入预训练好的词向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置
save_path = '../data/'

if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建

# ##################### config ######################
# title 部分 attention-bigru-512 
title_len = n_step1 = 30                  # title 长度
# content 部分 han-bigru-512
doc_len = 10                   # 每个文档的句子数
sent_len = 30                 # 每个句子的词数
# 其他参数
input_size = embedding_size = 256       # 字向量长度
n_class = 1999
hidden_size = 256    # 隐含层节点数
fc_hidden_size = 1024                   # fc 层节点数
title_attention_size = sent_attention_size = doc_attention_size = hidden_size * 2
n_layer = 1        # bi-gru 层数
max_grad_norm = 1.0  # 最大梯度（超过此值的梯度将被裁剪）
global_step = 0
valid_num = 100000
seed_num = 13
te_batch_size = 128 
tr_batch_size = 128 
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
Prepared, costed time 0.247077 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
HAN 模型，知乎问题多标签分类。
'''

print('Building model ...')
lr = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32, [])
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()   # BN 层中所有的更新操作


def weight_variable(shape, name):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

def gru_cell():
    with tf.variable_scope('gru_cell'):
        cell = rnn.GRUCell(hidden_size, reuse=tf.get_variable_scope().reuse)
    return rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
  
def bi_gru(inputs, seg_num=batch_size):
    """build the bi-GRU network. Return the encoder represented vector.
    n_step: 句子的词数量；或者文档的句子数。
    seg_num: 序列的数量，原本应该为 batch_size, 但是这里将 batch_size 个 doc展开成很多个句子。
    """
    cells_fw = [gru_cell() for _ in range(n_layer)]
    cells_bw = [gru_cell() for _ in range(n_layer)]
    initial_states_fw = [cell_fw.zero_state(seg_num, tf.float32) for cell_fw in cells_fw]
    initial_states_bw = [cell_bw.zero_state(seg_num, tf.float32) for cell_bw in cells_bw] 
    outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs, 
                        initial_states_fw = initial_states_fw, initial_states_bw = initial_states_bw, dtype=tf.float32)
    # outputs: Output Tensor shaped: seg_num, max_time, layers_output]，其中layers_output=hidden_size * 2 在这里。
    return outputs
    
    
def task_specific_attention(inputs, output_size,
                            initializer=layers.xavier_initializer(),
                            activation_fn=tf.tanh, scope=None):
    """
    Performs task-specific attention reduction, using learned
    attention context vector (constant within task of interest).
    Args:
        inputs: Tensor of shape [batch_size, units, input_size]
            `input_size` must be static (known)
            `units` axis will be attended over (reduced from output)
            `batch_size` will be preserved
        output_size: Size of output's inner (feature) dimension
    Returns:
        outputs: Tensor of shape [batch_size, output_dim].
    """
    assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
    with tf.variable_scope(scope or 'attention') as scope:
        # u_w, attention 向量
        attention_context_vector = tf.get_variable(name='attention_context_vector',
                                                   shape=[output_size],
                                                   initializer=initializer,
                                                   dtype=tf.float32)
        # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
        input_projection = layers.fully_connected(inputs, output_size,
                                                  activation_fn=activation_fn,
                                                  scope=scope)
        # 输出 [batch_size, units]
        vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
        attention_weights = tf.nn.softmax(vector_attn, dim=1)   
        tf.summary.histogram('attention_weigths', attention_weights)
        weighted_projection = tf.multiply(inputs, attention_weights)   # ???源代码,原文应该是对 hi 加权！！
        outputs = tf.reduce_sum(weighted_projection, axis=1)
        return outputs  # 输出 [batch_size, units]


with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, title_len], name='X1_inputs')
    X2_inputs = tf.placeholder(tf.int64, [None, doc_len*sent_len], name='X2_inputs')
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')    

with tf.name_scope('embedding'):
    embedding = tf.get_variable(name='embedding', shape=W_embedding.shape, 
                                initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune
    
def bigru_reference(X_inputs):
    """双端 gru 模型。title部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, title_len)
    Returns:
        title_outputs: tensor.shape=(batch_size, hidden_size(*2 for bigru))
    """
    inputs = tf.nn.embedding_lookup(embedding, X_inputs)
    with tf.variable_scope('title_encoder'):  # 生成 title 的向量表示
        title_outputs = bi_gru(inputs)
        title_attn_outputs = task_specific_attention(title_outputs, title_attention_size)   
    return title_attn_outputs   # shape = [batch_size, hidden_size*2]
    
def han_reference(X_inputs):
    """分层 attention 模型。content部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, doc_len*sent_len)
    Returns:
        doc_attn_outputs: tensor.shape=(batch_size, hidden_size(*2 for bigru))
    """
    inputs = tf.nn.embedding_lookup(embedding, X_inputs)    # inputs.shape=[batch_size, doc_len*sent_len, embedding_size]
    sent_inputs = tf.reshape(inputs,[batch_size*doc_len, sent_len, embedding_size]) # shape=(?, 40, 256)
    with tf.variable_scope('sentence_encoder'):  # 生成句向量
        sent_outputs = bi_gru(sent_inputs, seg_num=batch_size*doc_len)
        sent_attn_outputs = task_specific_attention(sent_outputs, sent_attention_size) # [batch_size*doc_len, hidden_size*2]
        with tf.variable_scope('dropout'):
            sent_attn_outputs = tf.nn.dropout(sent_attn_outputs, keep_prob)
    with tf.variable_scope('doc_encoder'):  # 生成句向量
        doc_inputs = tf.reshape(sent_attn_outputs, [batch_size, doc_len, hidden_size*2])
        doc_outputs = bi_gru(doc_inputs)  # [batch_size, doc_len, hidden_size*2]
        doc_attn_outputs = task_specific_attention(doc_outputs, doc_attention_size) # [batch_size, hidden_size*2]
#         with tf.variable_scope('dropout'):            # 不加 dropout
#             doc_attn_outputs = tf.nn.dropout(doc_attn_outputs, keep_prob)    
    return doc_attn_outputs    # [batch_size, hidden_size*2]
        

with tf.name_scope('title_bigru'):
    output_title = bigru_reference(X1_inputs)
with tf.name_scope('content_han'):
    output_content = han_reference(X2_inputs)   
    
with tf.name_scope('fc-bn-layer'):
    output = tf.concat([output_title, output_content], axis=1)
    W_fc = weight_variable([hidden_size*4, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[fc_hidden_size], name="beta_fc"))
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob)   # 不加 dropout
    
with tf.name_scope('out_layer'):
    W_out = weight_variable([fc_hidden_size, n_class], name='Weight_out') 
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  #每个类别的分数 scores
    
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)
    
# ***** 优化求解 *******
with tf.name_scope('AdamOptimizer'):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    train_op = optimizer.apply_gradients( zip(grads, tvars),
        global_step=tf.contrib.framework.get_or_create_global_step())
    update_op = tf.group(*update_emas)   # 更新 BN 参数

merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the han-bi-gru model.'

Building model ...
Finished creating the han-bi-gru model.


## 导入数据

In [3]:
import sys
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


print('loading data...')
time0 = time.time()
X_title = np.load(save_path+'X_tr_title_30.npy') 
sample_num = X_title.shape[0]
X_content = np.load(save_path+'train_segs30_content.npy')
X_content = X_content.reshape([-1, doc_len*sent_len])
X = np.hstack([X_title, X_content])
y = np.load(save_path+'y_tr.npy')
print('finished loading data, time cost %g s' % (time.time() - time0))
# 划分验证集
np.random.seed(seed_num)
new_index = np.random.permutation(sample_num)
X = X[new_index]
y = y[new_index]
X_valid = X[:valid_num]
y_valid = y[:valid_num]
X_train = X[valid_num:]
y_train = y[valid_num:]
print('train_num=%d, valid_num=%d' % (X_train.shape[0], X_valid.shape[0]))

# 构建数据生成器
data_train = BatchGenerator(X_train, y_train, shuffle=True)
data_valid = BatchGenerator(X_valid, y_valid, shuffle=False)
print('X_train.shape=', X_train.shape, 'y_train.shape=', y_train.shape)
print('X_valid.shape=', X_valid.shape, 'y_valid.shape=', y_valid.shape)

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
valid_data_size = data_valid.y.shape[0]

def valid_epoch():
    """Testing or valid."""
    data_valid._index_in_epoch = 0  # 先指向第一个值
    _batch_size = te_batch_size
    fetches = [cost, y_pred]   
    batch_num = int(valid_data_size / _batch_size)
    start_time = time.time()
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    for i in xrange(batch_num):
        X_batch, y_batch = data_valid.next_batch(_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5, 
                    batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost    
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / batch_num
    return mean_cost, precision, recall, f1

print('Finised loading data, time %g s' % (time.time() - time0))

loading data...
finished loading data, time cost 12.5653 s
train_num=2899952, valid_num=100000
('X_train.shape=', (2899952, 330), 'y_train.shape=', (2899952,))
('X_valid.shape=', (100000, 330), 'y_valid.shape=', (100000,))
Finised loading data, time 31.7651 s


## 训练模型

In [5]:
0.004 * (0.65**9)

8.284765135156253e-05

In [4]:
_lr = 2e-3
max_epoch = 1
max_max_epoch = 8  # 本例中，
decay = 0.65
valid_step = 7000        # 每 valid_step 就进行一次 valid 运算
save_epoch = 1           # 每迭代 save_epoch 次保存一次模型
tr_batch_num = int(data_train.y.shape[0] / tr_batch_size)  # 每个 epoch 中包含的 batch 数
saver = tf.train.Saver(max_to_keep=3)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.40
model_num = 0

for epoch in xrange(max_max_epoch):
    print 'EPOCH %d， lr=%g' % (epoch+1, _lr)
    time0 = time.time()
    _costs = 0.0
    for batch in xrange(tr_batch_num): 
        global_step += 1
        if (batch+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g ' % (
                    global_step, valid_cost, precision, recall, f1))
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path)
            _lr = _lr * decay
            print('===>_lr=%g' % _lr)
        X_batch, y_batch = data_train.next_batch(tr_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:_lr,
                    batch_size:tr_batch_size, keep_prob:0.5, tst:False, n_updates:global_step}
        fetches = [merged, cost, train_op, update_op]
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        _costs += _cost
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            X_batch, y_batch = data_valid.next_batch(tr_batch_size)
            X1_batch = X_batch[:, :title_len]
            X2_batch = X_batch[:, title_len:]
            y_batch = to_categorical(y_batch)
            feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5,
                         batch_size:tr_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)   

EPOCH 1， lr=0.002
Global_step=7000: valid cost=0.00486768; p=1.18499, r=0.484525, f1=0.343906 
===>_lr=0.002
Global_step=14000: valid cost=0.00438067; p=1.26718, r=0.517468, f1=0.367425 
===>_lr=0.002
Global_step=21000: valid cost=0.00425882; p=1.29724, r=0.529419, f1=0.375977 
===>_lr=0.002
EPOCH 2， lr=0.002
Global_step=29655: valid cost=0.00422456; p=1.30508, r=0.532378, f1=0.378129 
===>_lr=0.0013
Global_step=36655: valid cost=0.00410337; p=1.34728, r=0.548713, f1=0.389912 
===>_lr=0.000845
Global_step=43655: valid cost=0.00402182; p=1.36733, r=0.55681, f1=0.395679 
===>_lr=0.00054925
EPOCH 3， lr=0.00054925
Global_step=52310: valid cost=0.00399843; p=1.37859, r=0.561326, f1=0.398903 
===>_lr=0.000357013
Global_step=59310: valid cost=0.00398776; p=1.38427, r=0.563533, f1=0.400494 
('the save path is ', '../ckpt/m9-2-han-bigru-title-content-512-30/model.ckpt-1')
===>_lr=0.000232058
Global_step=66310: valid cost=0.00396555; p=1.38998, r=0.565989, f1=0.402211 
('the save path is ', '../

KeyboardInterrupt: 

In [6]:
valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
     valid_cost, precision, recall, f1, time.time()-time0) )

valid cost=0.00429876; p=1.36063, r=0.552703, f1=0.393044; speed=5141.38 s/epoch


In [8]:
save_path = saver.save(sess, model_path, global_step=(epoch+1))
print('the save path is ', save_path)

('the save path is ', '../ckpt/han-bigru-title-content-256/model.ckpt-4')


- Bi-LSTM 模型<br/>
**batch_size=256，迭代12个epoch，基本收敛。结果： 验证集 f1=0.38618； 提交结果 0.3873186**
- Bi-GRU 模型<br/>
**batch_size=256，迭代15个epoch。大概在 13 个 epoch 就收敛了。结果： 验证集 f1=0.390534； 提交结果 0.39198**
- attention-Bi-GRU 模型<br/>
**batch_size=256，迭代18个epoch。在16个epoch收敛。结果：验证集 f1=f1=0.391734 ；提交结果 0.39310**

## 若没有收敛，继续迭代

In [8]:
# 导入保存好的模型
time0 =time.time()
saver = tf.train.Saver()
best_model_path = model_path + '-' + str(7)  # 导入最优模型   f1=0.408237
saver.restore(sess, best_model_path)
print('Finished loading model.')
valid_cost, precision, recall, f1 = valid_epoch()
print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g ' % (
        global_step, valid_cost, precision, recall, f1))
print('costed time %g s' % (time.time()-time0))

INFO:tensorflow:Restoring parameters from ../ckpt/han-bigru-title-content-256-30/model.ckpt-7
Finished loading model.
Global_step=138321: valid cost=0.00388571; p=1.41074, r=0.574479, f1=0.408237 
costed time 109.284 s


In [7]:
model_num = 7
_lr = 3e-5
add_epoch = 3  # 本例中，
decay = 0.75
valid_step = 8000        # 每 valid_step 就进行一次 valid 运算
save_epoch = 1           # 每迭代 save_epoch 次保存一次模型
tr_batch_num = int(data_train.y.shape[0] / tr_batch_size)  # 每个 epoch 中包含的 batch 数
saver = tf.train.Saver(max_to_keep=5)           # 最多保存的模型数量
last_f1=0.408237
global_step = 79000

for epoch in xrange(add_epoch):
    _lr = _lr * decay
    print 'EPOCH %d， lr=%g' % (epoch+1, _lr)
    time0 = time.time()
    _costs = 0.0
    for batch in xrange(tr_batch_num): 
        global_step += 1
        if (batch+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g ' % (
                    global_step, valid_cost, precision, recall, f1))
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
            _lr = _lr * decay
            print('===>_lr=%g' % _lr)
        X_batch, y_batch = data_train.next_batch(tr_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:_lr,
                    batch_size:tr_batch_size, keep_prob:0.45, tst:False, n_updates:global_step}
        fetches = [merged, cost, train_op, update_op]
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        _costs += _cost
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            X_batch, y_batch = data_valid.next_batch(tr_batch_size)
            X1_batch = X_batch[:, :title_len]
            X2_batch = X_batch[:, title_len:]
            y_batch = to_categorical(y_batch)
            feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5,
                         batch_size:tr_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)
    valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
    mean_cost = _costs / tr_batch_num
    print('Global_step=%d; training cost=%g; valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
        global_step, mean_cost, valid_cost, precision, recall, f1, time.time()-time0) )
    if (f1 > last_f1):
        last_f1 = f1
        model_num += 1
        save_path = saver.save(sess, model_path, global_step=model_num)
        print('the save path is ', save_path)       

EPOCH 1， lr=2.25e-05
Global_step=87000: valid cost=0.00389798; p=1.40806, r=0.573273, f1=0.407403 
===>_lr=1.6875e-05
Global_step=95000: valid cost=0.00389498; p=1.40956, r=0.573765, f1=0.407778 
===>_lr=1.26562e-05
Global_step=101655; training cost=0.00361811; valid cost=0.00389419; p=1.40892, r=0.573452, f1=0.407567; speed=7553.93 s/epoch
EPOCH 2， lr=9.49219e-06
Global_step=109655: valid cost=0.00389483; p=1.4099, r=0.574043, f1=0.407947 
===>_lr=7.11914e-06
Global_step=117655: valid cost=0.00389145; p=1.41003, r=0.574021, f1=0.407946 
===>_lr=5.33936e-06
Global_step=124310; training cost=0.00360596; valid cost=0.00389107; p=1.41058, r=0.574252, f1=0.408109; speed=7648.16 s/epoch
EPOCH 3， lr=4.00452e-06
Global_step=132310: valid cost=0.00389246; p=1.40998, r=0.574214, f1=0.408039 
===>_lr=3.00339e-06


KeyboardInterrupt: 

## 本地测试

In [8]:
# sys.path.append('..')
# from evaluator import score_eval

# print('begin ...')
# # X_valid = np.load('../data/X_valid.npy')
# # 保存 本地测试的标注数据
# marked_labels_list = np.load('../data/marked_labels_list.npy')
# local_scores_path = '../local_scores/' + model_name + '.npy'

# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' + str(2)  # 导入最优模型
saver.restore(sess, best_model_path)
print('Finished loading model.')

INFO:tensorflow:Restoring parameters from ../ckpt/m9-han-bigru-title-content-512-30/model.ckpt-2
Finished loading model.


In [5]:
local_scores_path = '../local_scores/' + model_name + '.npy'
global_step = 800000
# 导入测试数据
def local_predict(scores_path=local_scores_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    X_te = X_valid
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in xrange(0, n_sample, _batch_size):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

local predicting ...
local valid p=1.39442, r=0.567602, f1=0.403398; speed=50.6908 s/epoch
Writed the scores into ../local_scores/m9-2-han-bigru-title-content-512-30.npy, time 51.115 s


## 对测试数据进行预测

In [15]:
# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' +str(7)
saver.restore(sess, best_model_path)

result_path = '../result/' + model_name + '-epoch4.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '-epoch4.npy'            # scores.npy 位置

INFO:tensorflow:Restoring parameters from ../ckpt/han-bigru-title-content-256/model.ckpt-4


In [6]:
save_path = '../data/'

In [7]:
# 导入测试数据
def predict():
    """预测测试集结果，并保存到  result.csv 文件中。"""
    X_title = np.load(save_path+'X_te_title_30.npy') 
    X_content = np.load(save_path+'eval_segs30_content.npy')
    X_content = X_content.reshape([-1, doc_len*sent_len])
    X_te = np.hstack([X_title, X_content])
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in xrange(0, n_sample, _batch_size):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5,
                    batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return predict_labels_list, np.vstack(predict_scores)

def write_result(predict_labels_list, result_path):
    """把结果写到 result.csv 中"""
    eval_question = np.load('../data/eval_question.npy')
    with open('../data/sr_topic2id.pkl', 'rb') as inp:
        sr_topic2id = pickle.load(inp)
        sr_id2topic = pickle.load(inp)
    pred_labels = np.asarray(predict_labels_list).reshape([-1])
    pred_topics = sr_id2topic[pred_labels].values.reshape([-1, 5])   # 转为 topic
    df_result = pd.DataFrame({'question':eval_question, 'tid0': pred_topics[:,0], 'tid1':pred_topics[:, 1],
                         'tid2': pred_topics[:,2], 'tid3':pred_topics[:,3],'tid4': pred_topics[:,4]})
    df_result.to_csv(result_path, index=False, header=False)
    print('Finished writing the result')
    return df_result

In [8]:
%time predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print('len(df_result)=',len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print('predict_scores.shape=',predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished saving the result!')

CPU times: user 1min 27s, sys: 20.6 s, total: 1min 47s
Wall time: 2min 2s
Finished writing the result
('len(df_result)=', 217360)
Saving the predict_scores into ../scores/m9-2-han-bigru-title-content-512-30.npy
('predict_scores.shape=', (217360, 1999))
Finished saving the result!


In [9]:
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,-7506384235581390893,2919247920214845195,-6306904715218704629
1,6649324930261961840,3418451812342379591,2858911571784840089,2382911985227044227,3383016985780045156,4313812860434517324
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-5265476641576484497,-7270992690764838239
3,6213817087034420233,-4966205278807386328,5804619920623030604,-8655945395761165989,244937959911721367,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-6162947324507053330,-1115593437686158905,-315342960042848847


In [12]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-7506384235581390893,-6306904715218704629,-6839713564940654454,2919247920214845195
1,6649324930261961840,-240041917918953337,2858911571784840089,3418451812342379591,-7483543763655495143,2382911985227044227
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-7270992690764838239,-5265476641576484497
3,6213817087034420233,-8655945395761165989,-4966205278807386328,5804619920623030604,7476760589625268543,244937959911721367
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,6018641953300645757,7951349602759061249


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
